# DSCI 100 Group 05 Project - Heart Disease

## Introduction

### Heart disease describes a range of conditions that affect the heart.


In [41]:
## Load libraries:
library(tidyverse)
library(readr)
library(dplyr)

## Load Heart Disease Data (name the object heart_disease)
cleveland_data <- read_delim("data/cleve.mod", delim = "\t", skip = 19, col_names = FALSE)

cleveland_data

Rows: 303 Columns: 1
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (1): X1

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


X1
<chr>
63.0 male angina 145.0 233.0 true hyp 150.0 fal 2.3 down 0.0 fix buff H
67.0 male asympt 160.0 286.0 fal hyp 108.0 true 1.5 flat 3.0 norm sick S2
67.0 male asympt 120.0 229.0 fal hyp 129.0 true 2.6 flat 2.0 rev sick S1
37.0 male notang 130.0 250.0 fal norm 187.0 fal 3.5 down 0.0 norm buff H
41.0 fem abnang 130.0 204.0 fal hyp 172.0 fal 1.4 up 0.0 norm buff H
56.0 male abnang 120.0 236.0 fal norm 178.0 fal 0.8 up 0.0 norm buff H
62.0 fem asympt 140.0 268.0 fal hyp 160.0 fal 3.6 down 2.0 norm sick S3
57.0 fem asympt 120.0 354.0 fal norm 163.0 true 0.6 up 0.0 norm buff H
63.0 male asympt 130.0 254.0 fal hyp 147.0 fal 1.4 flat 1.0 rev sick S2


In [53]:
## Clean data 

cleveland_data |> 
    separate(X1, into = c("age", "remove1","sex", "cp", "trestbps", "remove2", "chol", "remove3", "fbs", 
                   "restecg", "thalach", "remove4",  "exang", "oldpeak", "slope", 
                   "ca", "thal", "remove5","num", "remove6")) |>
    select(-c(remove1, remove2, remove3, remove4, remove5, remove6))


str(cleveland_data)
    


Warning message:
“Expected 20 pieces. Additional pieces discarded in 296 rows [1, 2, 3, 4, 5, 6,
7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”
Warning message:
“Expected 20 pieces. Missing pieces filled with `NA` in 5 rows [144, 158, 166,
251, 263].”


age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
63,male,angina,145,233,true,hyp,150,fal,2,3,down,0,fix
67,male,asympt,160,286,fal,hyp,108,true,1,5,flat,3,norm
67,male,asympt,120,229,fal,hyp,129,true,2,6,flat,2,rev
37,male,notang,130,250,fal,norm,187,fal,3,5,down,0,norm
41,fem,abnang,130,204,fal,hyp,172,fal,1,4,up,0,norm
56,male,abnang,120,236,fal,norm,178,fal,0,8,up,0,norm
62,fem,asympt,140,268,fal,hyp,160,fal,3,6,down,2,norm
57,fem,asympt,120,354,fal,norm,163,true,0,6,up,0,norm
63,male,asympt,130,254,fal,hyp,147,fal,1,4,flat,1,rev


spc_tbl_ [303 × 1] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ X1: chr [1:303] "63.0 male angina 145.0 233.0 true  hyp 150.0 fal  2.3 down 0.0  fix buff  H" "67.0 male asympt 160.0 286.0 fal   hyp 108.0 true 1.5 flat 3.0 norm sick S2" "67.0 male asympt 120.0 229.0 fal   hyp 129.0 true 2.6 flat 2.0  rev sick S1" "37.0 male notang 130.0 250.0 fal  norm 187.0 fal  3.5 down 0.0 norm buff  H" ...
 - attr(*, "spec")=
  .. cols(
  ..   X1 = col_character()
  .. )
 - attr(*, "problems")=<externalptr> 
